# Trademark Case Files Dataset

This dataset is found at https://www.uspto.gov/learning-and-resources/electronic-data-products/trademark-case-files-dataset-0.

The Trademark Case Files Dataset contains detailed information on 8.6 million trademark applications filed with or registrations issued by the USPTO between January 1870 and January 2017. It is derived from the USPTO main database for administering trademarks and includes data on mark characteristics, prosecution events, ownership, classification, third-party oppositions, and renewal history.

Full schema can be found at: https://www.uspto.gov/sites/default/files/documents/casefiles_schema_high_level_2016update.pdf

Data dictionary found at: https://www.uspto.gov/sites/default/files/documents/vartable_2016.pdf

For this analysis we'll only use the Case files and events table for exploring some details on existing and past trademark applications and registrations and their history of administrative actions taken from beginning to end.

More details on the full dataset can be found in this publication from the US Patent and Trademark office: http://papers.ssrn.com/sol3/papers.cfm?abstract_id=2188621

### Load our packages

In [1]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

In [2]:
import sqlContext.implicits._

In [3]:
import org.apache.spark.sql.types._

In [4]:
import org.apache.spark.sql.functions._

### Load our data
#### Case files data: 
Table which describe the current status of all trademark applications from 1870 - 2017 as of the time of the last data update (January 2017). 

In [5]:
val case_files_raw = spark.read.format("csv").option("header", "true").option("quote", "\"").option("escape", "\"").option("mode", "DROPMALFORMED").option("inferSchema", "true").load("hdfs://sandbox.hortonworks.com:8020/tmp/case_file.csv")

The case files data is quite large (over 8,000,000 records) and wide (79) variables. Let's just grab the columns we're interested in after reviewing the data schema here (https://www.uspto.gov/sites/default/files/documents/vartable_2016.pdf)

In [6]:
val case_files_cols = Seq("serial_no",
"abandon_dt",
"amend_reg_dt",
"reg_cancel_cd",
"reg_cancel_dt",
"cancel_pend_in",
"cert_mark_in",
"chg_reg_in",
"coll_memb_mark_in",
"coll_serv_mark_in",
"coll_trade_mark_in",
"serv_mark_in",
"draw_color_cur_in",
"draw_color_file_in",
"concur_use_in",
"concur_use_pend_in",
"filing_dt",
"for_priority_in",
"lb_itu_cur_in",
"lb_itu_file_in",
"interfer_pend_in",
"exm_office_cd",
"file_location_dt",
"mark_draw_cd",
"mark_id_char",
"opposit_pend_in",
"amend_principal_in",
"concur_use_pub_in",
"publication_dt",
"registration_dt",
"renewal_dt",
"renewal_file_in",
"cfh_status_cd",
"cfh_status_dt",
"trade_mark_in",
"registration_no")

In [7]:
val case_files_lite = case_files_raw.select(case_files_cols.head, case_files_cols.tail: _*)

Next, since this covers such wide range of time (almost 150 years of trademark registrations!), we want to look at trends over the years. We'll create year columns for all of the date columns since we'll access them a lot and don't want to regularly have to compute on the fly from the date columns. 

And while we are preparing some of the variables for easier analysis later, let's put the indicator for what type of trademark each case is (service, certification, or trademark) into one column since in the original dataset, it is spread across multiple columns with a boolean value in each column.

In [8]:
val case_files_lite_years_types = case_files_lite.
                            withColumn("filing_yr",year(case_files_lite("filing_dt"))).
                            withColumn("abandon_yr",year(case_files_lite("abandon_dt"))).
                            withColumn("publication_yr",year(case_files_lite("publication_dt"))).
                            withColumn("registration_yr",year(case_files_lite("registration_dt"))).
                            withColumn("cfh_status_yr",year(case_files_lite("cfh_status_dt"))).
                            withColumn("renewal_yr",year(case_files_lite("renewal_dt"))).
                            withColumn("reg_cancel_yr",year(case_files_lite("reg_cancel_dt"))).
                            withColumn("mark_type", when($"cert_mark_in" === 1, "certification").
                                                    when($"serv_mark_in" === 1, "service").
                                                    when($"trade_mark_in" === 1, "trademark").
                                                    otherwise("collective"))

In [12]:
case_files_lite_years_types.filter($"mark_type"==="trademark").count()

4715072

And let's then write out the derivative to a parquet file so we don't need to do this everytime

In [13]:
case_files_lite_years_types.write.parquet("hdfs://sandbox.hortonworks.com:8020/tmp/case_files_processed.parquet")

Then let's load it back in for faster traversal of the case files data

In [5]:
val case_files = sqlContext.read.parquet("hdfs://sandbox.hortonworks.com:8020/tmp/case_files_processed.parquet")

#### Events data:
Table which describes the event history of each trademark application from initiation through registration (or denial) and includes renewals, abandonments, and expiration.

In [14]:
val events_raw = spark.read.format("csv").option("header", "true").option("quote", "\"").option("escape", "\"").option("mode", "DROPMALFORMED").option("inferSchema", "true").load("hdfs://sandbox.hortonworks.com:8020/tmp/event.csv")

In [7]:
events_raw.columns

Array(event_cd, event_dt, event_seq, event_type_cd, serial_no)

### Question 1: Trends in types of  trademarks (service mark, trademark, certification mark, or collective mark)
- Ref: https://www.bitlaw.com/source/tmep/1306_01.html
- Check mark type for trends in service mark registrations like the dotcom boom
- Can we see an increase in trademark applications in general like the 1990s dotcom boom

In [8]:
val service_marks = case_files.filter($"serv_mark_in" === 1)

In [9]:
service_marks.count() //this is about a third of the total observations, as mentioned in the publication

2892932

In [10]:
val certification_marks = case_files.filter($"cert_mark_in" === 1)

In [11]:
certification_marks.count()

11893

In [12]:
val trademarks = case_files.filter($"trade_mark_in" === 1)

In [13]:
trademarks.count()

5154980

In [14]:
val service_counts = service_marks.filter($"filing_yr".isNotNull).groupBy("filing_yr").agg(count("*") as "serv_count").na.fill(0, Seq("serv_count")).orderBy($"filing_yr" asc)

In [15]:
val certification_counts = certification_marks.filter($"filing_yr".isNotNull).groupBy("filing_yr").agg(count("*") as "cert_count").na.fill(0, Seq("cert_count")).orderBy($"filing_yr" asc)

In [16]:
val trademark_counts = trademarks.filter($"filing_yr".isNotNull).groupBy("filing_yr").agg(count("*") as "trade_count").na.fill(0, Seq("trade_count")).orderBy($"filing_yr" asc)

In [17]:
val mark_type_counts_by_year = service_counts.join(certification_counts, Seq("filing_yr"), "left_outer").join(trademark_counts, Seq("filing_yr"), "left_outer")

In [18]:
mark_type_counts_by_year.orderBy($"filing_yr" asc).na.fill(0, Seq("cert_count")).show(150)

+---------+----------+----------+-----------+
|filing_yr|serv_count|cert_count|trade_count|
+---------+----------+----------+-----------+
|     1899|         1|         0|         46|
|     1911|         1|         0|        261|
|     1915|         1|         0|        278|
|     1931|         3|         0|       1023|
|     1932|         3|         0|        856|
|     1933|         1|         0|        975|
|     1934|         1|         0|       1141|
|     1935|         1|         0|       1034|
|     1937|         1|         0|        922|
|     1938|         2|         0|        928|
|     1939|         3|         1|        879|
|     1944|         1|         0|       1687|
|     1946|         4|         3|       2594|
|     1947|       170|         6|       6915|
|     1948|       142|         5|       5301|
|     1949|        87|         5|       3731|
|     1950|        86|         3|       3542|
|     1951|        82|         0|       3088|
|     1952|       115|         6| 

Pretty interesting. The original authors noticed a spike in service marks leading up to the dotcom boom with a noticeable drop off in 2001 thorugh 2004. Since 2005, we're seeing service marks steadily rise again, only dipping perhaps trivially after the economic crash of Oct 2008, only to steadily rise to historic highs in 2015 and 2016.

Compared to service marks, it's interesting to note that trademarks don't seem to have as noticeble interruption in numbers year over year, except for a slight drop in 2009, which again we might hypothesize was a result of the 2008 crash

In [ ]:
//lets save that table out to csv so we can visualize later
mark_type_counts_by_year.write.csv("hdfs://sandbox.hortonworks.com:8020/tmp/output/mark_type_counts_by_year.csv")

### Question 2: Getting a little more indepth and looking at the bigger picture of opposition to trademark applications
- Do oppositions increase over time as more and more trademarks are registered, or do the stay pretty proportional?
    - Service mark, trademark, certification mark
    - Bucket and aggregate by year
    - When were oppositions generated?
    - When were most oppositions sustained?


Events are logged in the event table. Of the 799 different event statuses (see https://eipweb.uspto.gov/TrademarkCaseFileEconomics/2011/event_description.csv.zip), we want to examine those involving oppositions, or rather, the action, after a trademark is accepted by the examiner and after it is "Published for opposition" for third parties to challenge that trademark before it is registered. There are 6 statuses we'll look at from the event_description file, 4 of which relate to opposing trademark applications:


| event_code | event_type | event_desc                                     | event_count |
|------------|------------|------------------------------------------------|-------------|
| PUBO       | A          | PUBLISHED FOR OPPOSITION                       | 5399155     |
| R.PR       | A          | REGISTERED-PRINCIPAL REGISTER                  | 3852684     |
| OP.I       | T          | OPPOSITION INSTITUTED NO. 999999               | 147676      |
| OP.T	     | T 	      | OPPOSITION TERMINATED NO. 999999	           | 144702      |
| OP.D       | T          | OPPOSITION DISMISSED NO. 999999                | 78278       |
| OP.S       | T          | OPPOSITION SUSTAINED NO. 999999                | 61363       |

After **PUBLISHED FOR OPPOSITION** the next major event to indicate that an opposition has actually been made is **OPPOSITION INSTITUTED**. Of that subset, the two major are outcomes are either **OPPOSITION DISMISSED** (and it can go on to be registered) or **OPPOSITION SUSTAINED** (and it will go on to be abandoned). There is a third status in **OPPOSITION TERMINATED**, either because the registering party did not respond to the opposition, or because the registering party filed a motion in response to the opposition and the opposing party failed to respond. More details on Trademark opposition proceedings and the Trademark Trial and Appeal Board (TTAB) can be found here: http://www.wipo.int/export/sites/www/sct/en/comments/pdf/sct17/us_1.pdf

*Note*: As there are so many different event codes and many different ways for an application to move through the trademark application process, there are events we are not accounting for in this analysis. With more time and understanding of the trademark process we'd be able to address those additional factors. But for this analysis, that we are focusing on the majority of outcomes related to opposing a trademark registration, this is at least somewhat informative.

**And so we ask**: Over time, when were opposition instituted most prevelant, and when were there more or less oppositions sustained (as related to the number of applications published for opposition and registered principal register by year)? 

**First**, lets reduce the amount of work the cluster has to do by only getting the events records related to the 4 opposition event codes `OP.I`, `OP.T`, `OP.D`, or `OP.S`.

In [15]:
val filtered_events = events_raw.filter($"event_cd" === "OP.I" || $"event_cd" === "OP.T" || $"event_cd" === "OP.D" || $"event_cd" === "OP.S" || $"event_cd" === "PUBO" || $"event_cd" === "R.PR").withColumn("event_yr", year($"event_dt"))

In [16]:
filtered_events.count()

9683862

In [17]:
filtered_events.show()

+--------+--------------------+---------+-------------+---------+--------+
|event_cd|            event_dt|event_seq|event_type_cd|serial_no|event_yr|
+--------+--------------------+---------+-------------+---------+--------+
|    R.PR|1997-02-09 00:00:...|        1|            A| 70029573|    1997|
|    R.PR|1901-01-01 00:00:...|        1|            A| 70035691|    1901|
|    R.PR|1901-07-23 00:00:...|        1|            A| 70036799|    1901|
|    R.PR|1903-11-03 00:00:...|        1|            A| 70041402|    1903|
|    R.PR|1904-11-22 00:00:...|        1|            A| 70043745|    1904|
|    R.PR|1904-11-29 00:00:...|        1|            A| 70043779|    1904|
|    R.PR|1905-01-31 00:00:...|        1|            A| 70044082|    1905|
|    R.PR|1905-03-07 00:00:...|        1|            A| 70044294|    1905|
|    OP.D|1982-02-16 00:00:...|        1|            T| 70073723|    1982|
|    OP.D|1983-02-18 00:00:...|        1|            T| 70082894|    1983|
|    OP.D|1982-11-22 00:0

At this point, it seems like having a lot of data loaded up is causing an issue for performace. Let's save the filtered events table and pick up from there when we re-run.

In [18]:
filtered_events.write.parquet("hdfs://sandbox.hortonworks.com:8020/tmp/oppose_events.parquet")

In [19]:
val oppose_events = sqlContext.read.parquet("hdfs://sandbox.hortonworks.com:8020/tmp/oppose_events.parquet")

**Second**, we want to make sure we can join the type to the event that took place based on serial number. So let's do that. As we recall, we created a column in the case_files table such that this would be easier to join together by serial number.

In [20]:
val case_files_types = case_files.select("serial_no", "mark_type")

In [21]:
val events_with_types = oppose_events.join(case_files_types, Seq("serial_no"), "left_outer")

In [22]:
events_with_types.show()

+---------+--------+--------------------+---------+-------------+--------+----------+
|serial_no|event_cd|            event_dt|event_seq|event_type_cd|event_yr| mark_type|
+---------+--------+--------------------+---------+-------------+--------+----------+
| 70373775|    OP.I|1983-07-12 00:00:...|        1|            T|    1983|collective|
| 72420637|    R.PR|1975-01-21 00:00:...|        1|            A|    1975| trademark|
| 73014197|    OP.D|1977-01-27 00:00:...|        1|            T|    1977| trademark|
| 73040412|    R.PR|1982-10-26 00:00:...|        3|            A|    1982| trademark|
| 73040412|    PUBO|1982-08-03 00:00:...|        2|            A|    1982| trademark|
| 73051380|    OP.D|1978-08-09 00:00:...|        1|            T|    1978|collective|
| 73067094|    OP.D|1978-03-21 00:00:...|        1|            T|    1978| trademark|
| 73071159|    OP.S|1978-01-03 00:00:...|        1|            T|    1978|collective|
| 73072014|    OP.D|1977-07-07 00:00:...|        1|   

In [23]:
events_with_types.write.parquet("hdfs://sandbox.hortonworks.com:8020/tmp/oppose_events_types.parquet")

In [24]:
val oppose_events_with_types = sqlContext.read.parquet("hdfs://sandbox.hortonworks.com:8020/tmp/oppose_events_types.parquet")

**Third**: Now that we have a neat, enriched subset of our data to play with for the final analysis, lets do some calculations to see when the most relative oppositions were over time.

In [31]:
val grouped_counts = oppose_events_with_types.groupBy("event_cd", "event_yr", "mark_type").agg(count("*").alias("counts")).orderBy($"event_yr" asc)

In [34]:
grouped_counts.show()

+--------+--------+---------+------+
|event_cd|event_yr|mark_type|counts|
+--------+--------+---------+------+
|    R.PR|    1901|trademark|     2|
|    R.PR|    1903|trademark|     1|
|    R.PR|    1904|trademark|     2|
|    R.PR|    1905|trademark|     3|
|    R.PR|    1906|trademark|     1|
|    R.PR|    1908|trademark|     1|
|    R.PR|    1911|trademark|     1|
|    R.PR|    1914|trademark|     1|
|    R.PR|    1916|trademark|     2|
|    R.PR|    1917|trademark|     1|
|    R.PR|    1919|trademark|     1|
|    R.PR|    1920|trademark|     1|
|    R.PR|    1921|trademark|     1|
|    R.PR|    1922|trademark|     1|
|    R.PR|    1923|trademark|     1|
|    OP.I|    1923|trademark|     1|
|    R.PR|    1924|trademark|     2|
|    R.PR|    1925|trademark|     1|
|    R.PR|    1927|trademark|     1|
|    R.PR|    1930|trademark|     2|
+--------+--------+---------+------+
only showing top 20 rows



In [33]:
grouped_counts.filter($"event_cd" === "OP.I").show()

+--------+--------+----------+------+
|event_cd|event_yr| mark_type|counts|
+--------+--------+----------+------+
|    OP.I|    1923| trademark|     1|
|    OP.I|    1965| trademark|     1|
|    OP.I|    1972|collective|     1|
|    OP.I|    1975|collective|     2|
|    OP.I|    1976| trademark|     6|
|    OP.I|    1976|collective|     3|
|    OP.I|    1976|   service|     2|
|    OP.I|    1977|collective|   130|
|    OP.I|    1977|   service|    10|
|    OP.I|    1977| trademark|    91|
|    OP.I|    1978| trademark|    56|
|    OP.I|    1978|collective|    77|
|    OP.I|    1978|   service|    15|
|    OP.I|    1979| trademark|    91|
|    OP.I|    1979|   service|    31|
|    OP.I|    1979|collective|    51|
|    OP.I|    1980|   service|    28|
|    OP.I|    1980|collective|    63|
|    OP.I|    1980| trademark|    93|
|    OP.I|    1981|collective|     3|
+--------+--------+----------+------+
only showing top 20 rows



### Question 3: length of marks over time

In looking over some of these trademarks, I noticed some like: "" & "". Just for fun, I wanted to get a sense for whether trademarks have gotten shorter over time, or if they've always had a similar average wordiness.

In [ ]:
val case_mark_lengths = case_files.filter($"filing_yr".isNotNull && $"mark_id_char".isNotNull).select("mark_id_char", "filing_yr").withColumn("count", size(split($"mark_id_char", " "))).orderBy($"filing_yr" asc)

In [ ]:
val avg_mark_counts = case_mark_lengths.groupBy("filing_yr").agg(mean("count")).orderBy($"filing_yr" asc)

In [ ]:
avg_mark_counts.write.csv("hdfs://sandbox.hortonworks.com:8020/tmp/output/avg_mark_counts.csv")